<a href="https://colab.research.google.com/github/Madhuri122002/Mail-Guardian-Shecodes/blob/main/cyber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [9]:
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from bs4 import BeautifulSoup
import re
import PyPDF2
from google.colab import files

def is_safe_url(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.head(url, headers=headers)
        if response.status_code == 200:
            return "This is a Safe URL"  # URL is safe
        else:
            return "This is NOT a Safe URL "# URL is potentially safe
    except requests.exceptions.ConnectionError:
        return "This is NOT a Safe URL"  # Unable to determine safety due to connection error
    except requests.exceptions.Timeout:
        return "This is NOT a Safe URL " # Unable to determine safety due to timeout
    except requests.exceptions.RequestException:
        return "This is NOT a Safe URL "  # Unable to determine safety due to other request exceptions

def analyze_attachments(email):
    attachments = [
        {'filename': 'document.pdf', 'type': 'PDF'},
        {'filename': 'spreadsheet.xlsx', 'type': 'Excel'},
        {'filename': 'malicious.exe', 'type': 'Executable'}
    ]
    attachment_results = []
    for attachment in attachments:
        if attachment['type'] in ['Executable', 'Script']:
            result = f"Attachment '{attachment['filename']}' is flagged as potentially malicious (type: {attachment['type']})"
        else:
            result = f"Attachment '{attachment['filename']}' is considered safe (type: {attachment['type']})"
        attachment_results.append(result)
    combined_result = "\n".join(attachment_results)
    return combined_result

def check_domain_reputation(domain, api_key):
    url = f'https://www.virustotal.com/vtapi/v2/domain/report'
    params = {'domain': domain, 'apikey': api_key}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        result = response.json()
        if 'detected_urls' in result:
            detected_urls = result['detected_urls']
            if len(detected_urls) > 0:
                return f"Domain '{domain}' has been flagged as malicious by {len(detected_urls)} scanners."
            else:
                return f"Domain '{domain}' has not been flagged as malicious."
        else:
            return f"Could not retrieve reputation for domain '{domain}'."
    else:
        return f"Failed to query VirusTotal. Status code: {response.status_code}"

def analyze_file_content(file_content):
    spam_keywords = [
        'viagra', 'cialis', 'pharmacy', 'online pharmacy', 'prescription', 'medication',
    'free', 'offer', 'earn', 'income', 'money', 'cash', 'guarantee', 'cheap',
    'credit', 'loan', 'debt', 'mortgage', 'finance', 'investment', 'bank', 'interest',
    'winner', 'prize', 'congratulations', 'lottery', 'winning', 'cash prize', 'free gift',
    'million', 'billion', 'trillion', 'urgent', 'immediate', 'act now', 'limited time',
    'exclusive', 'secret', 'hidden', 'miracle', 'magic', 'weight loss', 'diet', 'fat',
    'exercise', 'fitness', 'bodybuilding', 'enhancement', 'enlargement', 'replica',
    'luxury', 'watch', 'rolex', 'prada', 'designer', 'discount', 'sale', 'clearance',
    'work from home', 'home based', 'online marketing', 'mlm', 'multi-level marketing',
    'earn money', 'extra income', 'fast cash', 'no cost', 'risk-free', 'unsecured',
    'credit card', 'tracking', 'dollar', 'profit', 'guaranteed', 'save big money',
    'mortgage rates', 'lower rates', 'refinance', 'prevent', 'lose', 'weight', 'pounds',
    'click below', 'below market', 'shopper', 'order', 'direct email', 'bulk email', 'ad',
    'visit our website', "alert", "authenticate", "blocked", "breach", "click", "confirm",
    "credentials", "download", "error", "expired", "failure", "hack", "install",
    "invoice", "login", "malicious", "malware", "notification", "password", "phish",
    "reactivate", "refund", "risk", "secure", "security", "sign-in", "suspended",
    "threat", "trojan", "unauthorized", "unusual", "update", "verify", "virus",
    "warning", "account", "activation", "attachment", "compromise", "confidential",
    "danger", "deactivate", "deceptive", "encryption", "exploit", "fake", "fraud",
    "hijack", "identity", "infected", "keylogger", "link", "locked", "overdue",
    "payment", "phishing", "prompt", "protection", "reset", "reward", "scam",
    "sensitive", "server", "service", "spam", "spyware", "ssn", "steal",
    "subscription", "suspicious", "transaction", "unverified", "webmail", "win",
    "wire", "attempt", "block", "card"]

    for keyword in spam_keywords:
        if re.search(fr'\b{re.escape(keyword)}\b', file_content, flags=re.IGNORECASE):
            return "Content Analysis: This file is most probably classified as phishing/malicious mail based on content analysis."
    return "Content Analysis: This file is NOT classified as phishing/malicious mail based on content analysis."


def analyze_content(email_content):
    spam_keywords = [
        'viagra', 'cialis', 'pharmacy', 'online pharmacy', 'prescription', 'medication',
    'free', 'offer', 'earn', 'income', 'money', 'cash', 'guarantee', 'cheap',
    'credit', 'loan', 'debt', 'mortgage', 'finance', 'investment', 'bank', 'interest',
    'winner', 'prize', 'congratulations', 'lottery', 'winning', 'cash prize', 'free gift',
    'million', 'billion', 'trillion', 'urgent', 'immediate', 'act now', 'limited time',
    'exclusive', 'secret', 'hidden', 'miracle', 'magic', 'weight loss', 'diet', 'fat',
    'exercise', 'fitness', 'bodybuilding', 'enhancement', 'enlargement', 'replica',
    'luxury', 'watch', 'rolex', 'prada', 'designer', 'discount', 'sale', 'clearance',
    'work from home', 'home based', 'online marketing', 'mlm', 'multi-level marketing',
    'earn money', 'extra income', 'fast cash', 'no cost', 'risk-free', 'unsecured',
    'credit card', 'tracking', 'dollar', 'profit', 'guaranteed', 'save big money',
    'mortgage rates', 'lower rates', 'refinance', 'prevent', 'lose', 'weight', 'pounds',
    'click below', 'below market', 'shopper', 'order', 'direct email', 'bulk email', 'ad',
    'visit our website', "alert", "authenticate", "blocked", "breach", "click", "confirm",
    "credentials", "download", "error", "expired", "failure", "hack", "install",
    "invoice", "login", "malicious", "malware", "notification", "password", "phish",
    "reactivate", "refund", "risk", "secure", "security", "sign-in", "suspended",
    "threat", "trojan", "unauthorized", "unusual", "update", "verify", "virus",
    "warning", "account", "activation", "attachment", "compromise", "confidential",
    "danger", "deactivate", "deceptive", "encryption", "exploit", "fake", "fraud",
    "hijack", "identity", "infected", "keylogger", "link", "locked", "overdue",
    "payment", "phishing", "prompt", "protection", "reset", "reward", "scam",
    "sensitive", "server", "service", "spam", "spyware", "ssn", "steal",
    "subscription", "suspicious", "transaction", "unverified", "webmail", "win",
    "wire", "attempt", "block", "card"]

    for keyword in spam_keywords:
        if re.search(fr'\b{re.escape(keyword)}\b', email_content, flags=re.IGNORECASE):
            return "Content Analysis: This email is most probably classified as phishing/malicious mail based on content analysis."
    return "Content Analysis: This email is NOT classified as phishing/malicious mail based on content analysis."

# Placeholder for actual dataset path
data = pd.read_csv('spam.csv')
data['Spam'] = data['Category'].apply(lambda x: 1 if x == 'spam' else 0)
X_train, X_test, y_train, y_test = train_test_split(data.Message, data.Spam, test_size=0.25)

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])
clf.fit(X_train, y_train)


def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text


def predict_spam(email):
    attachment_result = analyze_attachments(email)
    sender_email = "ronak@gmail.com"
    api_key = "a57649402d6ce20f5f2c955b772726d00c1d1603c28189ab6930eb8d4acb6b3b"
    sender_reputation_result = check_domain_reputation(sender_email, api_key)
    content_result = analyze_content(email)
    safe_unsafe = is_safe_url(email)
    combined_result = f"Attachment Analysis: {attachment_result}\n" \
                      f"Domain Reputation Analysis: {sender_reputation_result}\n" \
                      f"url: {safe_unsafe}\n" \
                      f"{content_result}\n"
    prediction = clf.predict([email])
    if prediction[0] == 1:
        y_pred = clf.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy Score: {accuracy-0.15}")
        return f"{combined_result}This email is most probably classified as phishing/malicious mail."

    else:
        y_pred = clf.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy Score: {accuracy-0.15}")
        return f"{combined_result}This email is NOT classified as phishing/malicious mail."


email_input = input("Enter the email content: ")
result = predict_spam(email_input)
print(result)


print("Select the PDF file:")
uploaded = files.upload()

if not uploaded:
  print("No files selected")
else:
  for pdf_path in uploaded.keys():
    extracted_text = extract_text_from_pdf(pdf_path)
    content_analysis=analyze_file_content(extracted_text)
    print("Content Analysis: ",content_analysis)

Enter the email content: Dear Debbie!  I hope this email reaches you in good health. My name is Samantha West, contacting you urgently on behalf of SuperDigflashes. We've been closely monitoring your outstanding contributions in Marketing, and we need your immediate expertise in a time-sensitive matter.  We have identified you as one of the few professionals capable of addressing the imminent challenges in  "challenges marketers face while implementing AI". You are personally invited to an webinar  scheduled for 21 January, 2024, where industry leaders will discuss critical developments.  To ensure your spot, click on the following link NOW: https:superdigflashes-register.com. We understand the urgency of your insights and expect your swift confirmation.  Please note that failure to attend may result in missed opportunities for your career growth. As a token of our gratitude, participants will receive an exclusive e-book containing confidential industry strategies.  We appreciate your 

No files selected
